A notebook that allows manual interactions with an RTSP server.  
Intended to increase one's understanding of the RTSP protocol.

In [1]:
import socket
import hashlib
from urllib.parse import urlparse

In [2]:
# Function to read RTSP URLs from a file
def read_rtsp_urls(filename):
    urls = []
    try:
        with open(filename, 'r') as file:
            urls = [line.strip() for line in file.readlines() if line.strip()]
    except FileNotFoundError:
        print(f"Error: File {filename} not found.")
    return urls

In [11]:
class RTSPClient:
    def __init__(self, url):
        self.parsed_url = urlparse(url)
        self.url = self.remove_credentials_from_url()
        self.server_ip = self.parsed_url.hostname
        self.server_port = self.parsed_url.port if self.parsed_url.port else 554
        self.username = self.parsed_url.username
        self.password = self.parsed_url.password
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.session_id = None
        self.auth_realm = None
        self.auth_nonce = None
        self.auth_required = False
    
    def connect(self):
        try:
            self.client_socket.connect((self.server_ip, self.server_port))
        except Exception as e:
            print(f"Error connecting to server: {e}")

    def close(self):
        self.client_socket.close()

    def remove_credentials_from_url(self):
        # Reconstruct the URL without username and password
        new_url = self.parsed_url._replace(netloc=self.parsed_url.hostname + (f":{self.parsed_url.port}" if self.parsed_url.port else ""))
        return new_url.geturl()

    def send_rtsp_options(self):
        try:
            # Prepare the RTSP OPTIONS request
            cseq = 1
            options_request = f"OPTIONS {self.remove_credentials_from_url()} RTSP/1.0\r\n"
            options_request += f"CSeq: {cseq}\r\n"
            options_request += "User-Agent: Python RTSP Client\r\n"
            options_request += "\r\n"
            
            # Send the request to the RTSP server
            self.client_socket.send(options_request.encode())
            
            # Receive the response from the server
            response = self.client_socket.recv(4096)
            print("RTSP Server Response (OPTIONS):\n")
            print(response.decode())
            
        except Exception as e:
            print(f"Error: {e}")

    def send_rtsp_describe(self):
        try:
            # Prepare the initial RTSP DESCRIBE request
            cseq = 2
            describe_request = f"DESCRIBE {self.remove_credentials_from_url()} RTSP/1.0\r\n"
            describe_request += f"CSeq: {cseq}\r\n"
            describe_request += "User-Agent: Python RTSP Client\r\n"
            describe_request += "Accept: application/sdp\r\n"
            describe_request += "\r\n"
            
            # Send the request to the RTSP server
            self.client_socket.send(describe_request.encode())
            
            # Receive the response from the server
            response = self.client_socket.recv(4096)
            response_str = response.decode()
            print("RTSP Server Response (DESCRIBE):\n")
            print(response_str)
            
            # Extract session ID if available
            self.session_id = None
            for line in response_str.split("\r\n"):
                if line.startswith("Session:"):
                    self.session_id = line.split(" ")[1]
                    break
            
            # Check if authentication is required (401 Unauthorized)
            if "401 Unauthorized" in response_str:
                self.auth_required = True
                # Extract the realm and nonce from the response
                self.auth_realm = None
                self.auth_nonce = None
                for line in response_str.split("\r\n"):
                    if line.startswith("WWW-Authenticate: Digest"):
                        parts = line.split(",")
                        for part in parts:
                            if "realm" in part:
                                self.auth_realm = part.split('"')[1]
                            elif "nonce" in part:
                                self.auth_nonce = part.split('"')[1]
                
                if self.auth_realm and self.auth_nonce and self.username and self.password:
                    # Compute the digest response
                    ha1 = hashlib.md5(f"{self.username}:{self.auth_realm}:{self.password}".encode()).hexdigest()
                    ha2 = hashlib.md5(f"DESCRIBE:{self.remove_credentials_from_url()}".encode()).hexdigest()
                    response_digest = hashlib.md5(f"{ha1}:{self.auth_nonce}:{ha2}".encode()).hexdigest()
                    
                    # Prepare the authenticated DESCRIBE request
                    cseq += 1
                    describe_request = f"DESCRIBE {self.remove_credentials_from_url()} RTSP/1.0\r\n"
                    describe_request += f"CSeq: {cseq}\r\n"
                    describe_request += "User-Agent: Python RTSP Client\r\n"
                    describe_request += "Accept: application/sdp\r\n"
                    describe_request += (
                        f"Authorization: Digest username=\"{self.username}\", realm=\"{self.auth_realm}\", nonce=\"{self.auth_nonce}\", uri=\"{self.remove_credentials_from_url()}\", response=\"{response_digest}\"\r\n"
                    )
                    describe_request += "\r\n"
                    
                    # Send the authenticated request to the RTSP server
                    self.client_socket.send(describe_request.encode())
                    
                    # Receive the response from the server
                    response = self.client_socket.recv(4096)
                    response_str = response.decode()
                    print("RTSP Server Response (Authenticated DESCRIBE):\n")
                    print(response_str)
                    
                    # Extract session ID if available
                    for line in response_str.split("\r\n"):
                        if line.startswith("Session:"):
                            self.session_id = line.split(" ")[1]
                            break
                else:
                    print("Authentication required, but realm/nonce extraction failed or credentials not provided.")
            
        except Exception as e:
            print(f"Error: {e}")

    def send_rtsp_setup(self):
        try:
            # Prepare the RTSP SETUP request
            cseq = 3
            setup_request = f"SETUP {self.remove_credentials_from_url()}&trackID=0 RTSP/1.0\r\n"
            setup_request += f"CSeq: {cseq}\r\n"
            setup_request += "Transport: RTP/AVP;unicast;client_port=8000-8001\r\n"
            setup_request += "User-Agent: Python RTSP Client\r\n"
            if self.session_id:
                setup_request += f"Session: {self.session_id}\r\n"
            
            # If authentication is required, add the Authorization header
            if self.auth_required and self.auth_realm and self.auth_nonce and self.username and self.password:
                # Compute the digest response
                ha1 = hashlib.md5(f"{self.username}:{self.auth_realm}:{self.password}".encode()).hexdigest()
                ha2 = hashlib.md5(f"SETUP:{self.remove_credentials_from_url()}&trackID=0".encode()).hexdigest()
                response_digest = hashlib.md5(f"{ha1}:{self.auth_nonce}:{ha2}".encode()).hexdigest()
                setup_request += (
                    f"Authorization: Digest username=\"{self.username}\", realm=\"{self.auth_realm}\", nonce=\"{self.auth_nonce}\", uri=\"{self.remove_credentials_from_url()}&trackID=0\", response=\"{response_digest}\"\r\n"
                )
            setup_request += "\r\n"
            
            # Send the SETUP request
            self.client_socket.send(setup_request.encode())
            
            # Receive the response from the server
            response = self.client_socket.recv(4096)
            response_str = response.decode()
            print("RTSP Server Response (SETUP):\n")
            print(response_str)
            
        except Exception as e:
            print(f"Error: {e}")


In [5]:
rtsp_urls = read_rtsp_urls('rtsp_urls.txt')

In [13]:
client = RTSPClient(rtsp_urls[1][1:-1])
client.connect()
client.send_rtsp_options()
client.send_rtsp_describe()
client.send_rtsp_setup()

#send_rtsp_options(client_socket, url_with_no_cred)
##send_rtsp_describe(client_socket, url_with_no_cred, username, password)
#send_rtsp_setup(client_socket, url_with_no_cred, username, password)

RTSP Server Response (OPTIONS):

RTSP/1.0 200 OK
CSeq: 1
Public: OPTIONS, DESCRIBE, PLAY, PAUSE, SETUP, TEARDOWN, SET_PARAMETER, GET_PARAMETER
Date: Sun, 27 Oct 2024 18:58:49 GMT


RTSP Server Response (DESCRIBE):

RTSP/1.0 401 Unauthorized
CSeq: 2
WWW-Authenticate: Digest realm="device",nonce="fb2e1b98596706f2e7641d8c740b326de45b0c1a41e55cb86dcfe6290def9d59"
Date: Sun, 27 Oct 2024 18:58:49 GMT


RTSP Server Response (Authenticated DESCRIBE):

RTSP/1.0 200 OK
CSeq: 3
Content-Type: application/sdp
Content-Length: 402
Date: Sun, 27 Oct 2024 18:58:49 GMT

v=0
o=- 68329799566 68329799566 IN IP4 192.168.50.60
s=RTSP Session
t=0 0
m=video 0 RTP/AVP 96
a=control:rtsp://192.168.50.60:554/rtsp/streaming?channel=3&subtype=1&trackID=1
a=rtpmap:96 H265/90000
a=framerate:6
a=x-dimensions:704,480
a=fmtp:96 sprop-vps=QAEMBP//IWAAAAMAAAMAAAMAAAMAewAArWsCQA==; sprop-sps=QgEEIWAAAAMAAAMAAAMAAAMAewAAoAWCAeH/rWtOSia5VZA=; sprop-pps=RAHAcvAbJA==
a=recvonly

RTSP Server Response (SETUP):

RTSP/1.0 401 Unaut

In [12]:
client.close()